#  Tunix Kaggle Hackathon - Environment Setup

This notebook sets up the environment for training Gemma reasoning models with Tunix on Kaggle TPUs.

**Time estimate:** ~20-30 minutes

## 1. Check TPU Availability

In [ ]:
import os
import sys

# Check if running on Kaggle
ON_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
print(f"Running on Kaggle: {ON_KAGGLE}")

# Check TPU
try:
    import jax
    print(f"JAX version: {jax.__version__}")
    print(f"Devices: {jax.devices()}")
    print(f"Device count: {jax.device_count()}")
except ImportError:
    print("JAX not installed yet - will install below")

## 2. Install Dependencies

In [ ]:
# Upgrade pip
!pip install --upgrade pip -q

# Install Tunix from GitHub
!pip install git+https://github.com/google/tunix.git -q

# Install other dependencies
!pip install flax optax transformers datasets accelerate -q
!pip install tensorboard pyyaml -q

print(" Dependencies installed!")

## 3. Verify Installations

In [ ]:
import jax
import jax.numpy as jnp
import flax
import optax
import transformers

print(f"JAX: {jax.__version__}")
print(f"Flax: {flax.__version__}")
print(f"Optax: {optax.__version__}")
print(f"Transformers: {transformers.__version__}")

# Verify TPU access
devices = jax.devices()
print(f"\n Available devices: {len(devices)}")
for d in devices:
    print(f"  - {d}")

In [ ]:
# Import Tunix
try:
    import tunix
    print(f"Tunix imported successfully!")
    print(f"Available modules: {dir(tunix)}")
except ImportError as e:
    print(f"Tunix import error: {e}")
    print("Try: pip install git+https://github.com/google/tunix.git")

## 4. Load Tokenizer

In [ ]:
from transformers import AutoTokenizer

# Load Gemma tokenizer
MODEL_NAME = "google/gemma-3-1b-it"  # or "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer loaded: {MODEL_NAME}")
print(f"Vocab size: {tokenizer.vocab_size}")

# Test tokenization
test_text = "<reasoning>Step 1: Calculate.</reasoning><answer>42</answer>"
tokens = tokenizer(test_text)
print(f"\nTest tokenization:")
print(f"  Input: {test_text}")
print(f"  Tokens: {len(tokens['input_ids'])} tokens")

## 5. Quick JAX Test

In [ ]:
# Quick matrix multiplication to verify TPU works
import jax.numpy as jnp
import time

# Create random matrices
key = jax.random.PRNGKey(42)
x = jax.random.normal(key, (1000, 1000))

# Warm up
_ = jnp.dot(x, x)

# Benchmark
start = time.time()
for _ in range(100):
    y = jnp.dot(x, x)
y.block_until_ready()
elapsed = time.time() - start

print(f" 100x matrix multiplications: {elapsed:.2f}s")
print(f"   (~{elapsed*10:.1f}ms per 1000x1000 matmul)")

## 6. Setup Project Directories

In [ ]:
import os

# Create directories
dirs = [
    'data/raw',
    'data/prepared', 
    'data/tokenized',
    'checkpoints/sft',
    'checkpoints/rl',
    'logs/tensorboard',
    'logs/eval_reports'
]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    print(f" Created: {d}")

print("\n Project structure ready!")

## 7. Session Info

In [ ]:
import datetime

print("=" * 50)
print("SESSION INFO")
print("=" * 50)
print(f"Time: {datetime.datetime.now()}")
print(f"Model: {MODEL_NAME}")
print(f"TPU devices: {jax.device_count()}")
print(f"Python: {sys.version}")
print("=" * 50)
print("\n Environment setup complete!")
print(" Proceed to: 01_data_prep_and_tokenize.ipynb")